# Observer Pattern

观察者模式

## Introduction

观察者模式是一种行为型设计模式，它定义了一种一对多的依赖关系，当一个对象的状态发生改变时，其所有依赖者都会收到通知并自动更新。

当对象间存在一对多关系时，则使用观察者模式（Observer Pattern）。比如，当一个对象被修改时，则会自动通知依赖它的对象。观察者模式属于行为型模式。

```
   A
 / | \
B  C  D
```

A 发生了改变时，B C D 也要根据 A 的变化 **自动** 更新

关注重点：state

In [13]:
from abc import ABCMeta, abstractmethod

class Observer(metaclass=ABCMeta):
    @abstractmethod
    def update(self, observable, state):
        pass

class Observable:
    """维护一个列表存储所有的观察者"""
    def __init__(self) -> None:
        self._observer_list = []

    """添加"""
    def add_observer(self, observer):
        self._observer_list.append(observer)

    """删除"""
    def remove_observer(self, observer):
        if observer in self._observer_list:
            self._observer_list.remove(observer)

    """通知（触发）"""
    def notify(self, value):
        for observer in self._observer_list:
            observer.update(self, value)

## Example

简单的例子，比如热水器热水，到多少度可以饮用，到多少度可以洗澡的问题

In [14]:
class WaterHeater:
    def __init__(self) -> None:
        self._temperature = 20.0

    @property
    def temperature(self):
        return self._temperature
    
    @temperature.setter
    def temperature(self, value):
        if value > 101.0:
            raise ValueError(f"常压下烧不出 {value} 度的水，请找高压锅炉")
        
        self._temperature = value


def test_water_heater(water_heater):
    temperature = water_heater.temperature
    if temperature < 50.0:
        print(f"才 {temperature} 度，水还没烧起来")
    elif temperature < 100.0:
        print(f"{temperature} 度了，可以洗澡了")
    else:
        print(f"水开了，可以喝了")


"""
1. 设置水温
2. 检查是不是满足某些条件
"""
water_heater = WaterHeater()
print(f"初始水温：{water_heater.temperature} 度")

"""烧到 40 度"""
water_heater.temperature = 40.0
print(f"当前水温：{water_heater.temperature} 度")
test_water_heater(water_heater)

"""烧到 60 度"""
water_heater.temperature = 60.0
print(f"当前水温：{water_heater.temperature} 度")
test_water_heater(water_heater)

"""烧到 100 度"""
water_heater.temperature = 100.0
print(f"当前水温：{water_heater.temperature} 度")
test_water_heater(water_heater)

"""烧不了超过 100.0 度的水"""
water_heater.temperature = 114.514

初始水温：20.0 度
当前水温：40.0 度
才 40.0 度，水还没烧起来
当前水温：60.0 度
60.0 度了，可以洗澡了
当前水温：100.0 度
水开了，可以喝了


ValueError: 常压下烧不出 114.514 度的水，请找高压锅炉

## 思考？

上面的案例有什么问题？


## 用 Observer pattern 改进

In [18]:
class WaterHeater(Observable):
    def __init__(self) -> None:
        super().__init__()
        self._temperature = 20.0

    @property
    def temperature(self):
        return self._temperature
    
    @temperature.setter
    def temperature(self, value):
        if value > 101.0:
            raise ValueError(f"常压下烧不出 {value} 度的水，请找高压锅炉")
        
        self._temperature = value

        """在设置温度的时候触发 Observer"""
        self.notify(self.temperature)


class ShowerObserver(Observer):
    def update(self, water_heater, value):
        if isinstance(water_heater, WaterHeater) and 50.0 < water_heater.temperature < 100.0:
            print(f"{water_heater.temperature} 度了，可以洗澡了")


class DrinkingObserver(Observer):
    def update(self, water_heater, value):
        if isinstance(water_heater, WaterHeater) and water_heater.temperature >= 100.0:
            print(f"{water_heater.temperature} 度了，可以洗澡了")


"""被观察对象"""
water_heater = WaterHeater()

"""观察者"""
shower_observer = ShowerObserver()
drinking_observer = DrinkingObserver()

"""订阅"""
water_heater.add_observer(shower_observer)
water_heater.add_observer(drinking_observer)
print(f"Num of observers: {len(water_heater._observer_list)}")

"""开始烧水"""
print(f"初始水温：{water_heater.temperature} 度")

"""烧到 40 度"""
water_heater.temperature = 40.0
print(f"当前水温：{water_heater.temperature} 度")

"""烧到 60 度"""
water_heater.temperature = 60.0
print(f"当前水温：{water_heater.temperature} 度")

"""烧到 100 度"""
water_heater.temperature = 100.0
print(f"当前水温：{water_heater.temperature} 度")

"""烧不了超过 100.0 度的水"""
water_heater.temperature = 114.514

Num of observers: 2
初始水温：20.0 度
当前水温：40.0 度
60.0 度了，可以洗澡了
当前水温：60.0 度
100.0 度了，可以洗澡了
当前水温：100.0 度


ValueError: 常压下烧不出 114.514 度的水，请找高压锅炉

In [19]:
"""然后可以把 observer 清理掉"""
water_heater.remove_observer(shower_observer)
water_heater.remove_observer(drinking_observer)
print(f"Num of observers: {len(water_heater._observer_list)}")

Num of observers: 0


## 再多聊点（碎碎念）

这里的 Observer Pattern 是松耦合关系，Observer 直接 subscribe Subject

还有一种模式叫 发布-订阅模式（Publish-Subscribe Pattern）

![img](./observer-pattern.asserts/fig.png)

它们两个之间的区别在于 Observer 是怎么和 Subject 耦合在一起的。Publish-Subscribe Pattern 里会再多一层发布的 **消息** 的抽象

上面的热水器也可以改成 发布-订阅模式，没事可以自己试试

观察者模式也可以变成一个链条一直触发下去，但要小心别循环了

销毁被观察的对象时，记得移除所有的观察者，在 发布-订阅模式 中搞不好会把已经消息发给已经被销毁掉的对象

个人感觉观察者模式在前端开发这块用的会更多，前端很典型的一个链条就是

```
用户交互 -> 数据更新 -> 触发 UI 改变
```

这里 **数据** 是被观察的对象

Model-View-Controller (MVC), Model-ModelView-View (MVVM) 等